In [1]:
import os, sys, glob, shutil

In [2]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [3]:
import argparse
import yaml
import numpy as np
from pprint import pprint
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from utils.model import get_model, get_vocoder, get_param_num, vocoder_infer
from utils.tools import to_device, log, synth_one_sample, expand, plot_mel
from model import FastSpeech2Loss
from dataset import Dataset, VideoDataset
# from utils.auto_tqdm import tqdm
from evaluate import evaluate

In [4]:
from preprocessor.preprocessor import Preprocessor

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/Ego4D_final_v6/0719a_preprocess.yaml -m ./config/Ego4D_final_v6/0719a_model.yaml -t ./config/Ego4D_final_v6/0719a_train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())

In [7]:
args

Namespace(restore_step=0, preprocess_config='./config/Ego4D_final_v6/0719a_preprocess.yaml', model_config='./config/Ego4D_final_v6/0719a_model.yaml', train_config='./config/Ego4D_final_v6/0719a_train.yaml')

In [8]:
config = yaml.load(open(args.preprocess_config, "r"), Loader=yaml.FullLoader)
preprocessor = Preprocessor(config)

Video count: 194


In [9]:
# preprocessor.build_from_trainval_path()

In [10]:
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)

In [11]:
print("Prepare training ...")

preprocess_config, model_config, train_config = configs

# Get dataset
dataset = VideoDataset(
    "train.txt", 'train', preprocess_config, train_config, sort=True, drop_last=True
)

Prepare training ...


In [12]:
len(dataset)

27292

In [13]:
dataset.batch_size

24

In [14]:
batch_size = train_config["optimizer"]["batch_size"]
group_size = 4  # Set this larger than 1 to enable sorting in Dataset
assert batch_size * group_size < len(dataset)
loader = DataLoader(
    dataset,
    batch_size=batch_size * group_size,
    shuffle=True,
    collate_fn=dataset.collate_fn,
)

In [15]:
# return (
#     ids,
#     raw_texts,
#     speakers,
#     texts,
#     text_lens,
#     max(text_lens),
#     mels,
#     mel_lens,
#     max(mel_lens),
#     pitches,
#     energies,
#     durations,
#     speaker_embeddings,
#     video_embeddings,
#     vid_lens,
# )

In [16]:
# i = 1
# for data in loader:
#     i += 1
#     print(len(data))
#     if i>= 5:
#         break

In [17]:
# for item in data:
#     print(item[10].shape)

In [18]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)

# Load vocoder
vocoder = get_vocoder(model_config, device)

=> Using speaker embeddings.
=> Using VarianceAdaptorWithSpeaker.
Successfully loaded from ./output/LibriTTS/LibriTTS_800000.pth.tar
Number of FastSpeech2 Parameters: 35165553
Removing weight norm...


In [19]:
# Training
step = args.restore_step + 1
epoch = 1
grad_acc_step = train_config["optimizer"]["grad_acc_step"]
grad_clip_thresh = train_config["optimizer"]["grad_clip_thresh"]
total_step = train_config["step"]["total_step"]
log_step = train_config["step"]["log_step"]
save_step = train_config["step"]["save_step"]
synth_step = train_config["step"]["synth_step"]
val_step = train_config["step"]["val_step"]

In [20]:
for batchs in loader:
    for batch in batchs:
        batch = to_device(batch, device)
        temp_batch = (batch[2:])
        output = model(*(batch[2:]))
        break
    break

mel_lens:  tensor([1099,  618,  474,  412,  305,  778,  302,  300,  325,  299,  246,  378,
         319,  206,  184,  301,  138,  105,  259,  117,  212,  156,  226,  239],
       device='cuda:0', dtype=torch.int32)
mel_masks.shape:  torch.Size([24, 1099])
mel_masks.dtype:  torch.bool
texts.shape:  torch.Size([24, 134])
=> Retrieved text embeddings shape:  torch.Size([24, 134, 256])
=> Mask shape:  torch.Size([24, 134])
attn output shape: torch.Size([24, 134, 256])
1 torch.Size([24, 134, 256])
cnn output shape: torch.Size([24, 134, 256])
attn output shape: torch.Size([24, 134, 256])
1 torch.Size([24, 134, 256])
cnn output shape: torch.Size([24, 134, 256])
attn output shape: torch.Size([24, 134, 256])
1 torch.Size([24, 134, 256])
cnn output shape: torch.Size([24, 134, 256])
attn output shape: torch.Size([24, 134, 256])
1 torch.Size([24, 134, 256])
cnn output shape: torch.Size([24, 134, 256])
attn output shape: torch.Size([24, 1000, 256])
1 torch.Size([24, 1000, 256])
cnn output shape: to